In [1]:
map_names_not_same = {
                        "Alabama-Birmingham" : "UAB",
                        "Albany (NY)": "Albany",
                        "Arkansas-Pine Bluff" : "Arkansas Pine Bluff",
                        "Bethune-Cookman" : "Bethune Cookman",
                        "Bowling Green State" : "Bowling Green",
                        "Brigham Young" : "BYU",
                        "Cal State Bakersfield" : "Cal St. Bakersfield",
                        "Cal State Fullerton" : "Cal St. Fullerton",
                        "Cal State Northridge" : "Cal St. Northridge",
                        "California Baptist" : "Cal Baptist",
                        "UC-Davis" : "UC Davis",
                        "UC-Irvine" : "UC Irvine",
                        "UC-Riverside" : "UC Riverside",
                        "UCSB" : "UC Santa Barbara",
                        "University of California" : "California",
                        "Central Connecticut State" : "Central Connecticut",
                        "Central Florida" : "UCF",
                        "Citadel" : "The Citadel",
                        "College of Charleston" : "Charleston",
                        "Detroit Mercy" : "Detroit",
                        "Florida International" : "FIU",
                        "Gardner-Webb" : "Gardner Webb",
                        "Grambling" : "Grambling St.",
                        "Illinois-Chicago" : "Illinois Chicago",
                        "Purdue-Fort Wayne" : "Purdue Fort Wayne",
                        "Cal State Long Beach" : "Long Beach St.",
                        "Long Island University" : "LIU",
                        "Louisiana-Monroe" : "Louisiana Monroe",
                        "Louisiana State" : "LSU",
                        "Loyola (IL)" : "Loyola Chicago",
                        "Loyola (MD)" : "Loyola Marymount",
                        "Maryland-Baltimore County" : "UMBC",
                        "Maryland-Eastern Shore" : "Maryland Eastern Shore",
                        "Massachusetts-Lowell" : "UMass Lowell",
                        "Miami (FL)" : "Miami FL",
                        "Miami (OH)" : "Miami OH",
                        "Missouri-Kansas City" : "UMKC",
                        "Omaha" : "Nebraska Omaha",
                        "Nevada-Las Vegas" : "UNLV",
                        "North Carolina-Asheville" : "UNC Asheville",
                        "North Carolina-Greensboro" : "UNC Greensboro",
                        "North Carolina-Wilmington" : "UNC Wilmington",
                        "NC State" : "N.C. State",
                        "Pennsylvania" : "Penn",
                        "Prairie View" : "Prairie View A&M",
                        "Saint Francis (PA)" : "St. Francis PA",
                        "Saint Mary's (CA)" : "Saint Mary's",
                        "South Carolina Upstate" : "South Carolina St.",
                        "Southern California" : "USC",
                        "Southern Methodist" : "SMU",
                        "Southern Mississippi" : "Southern Miss",
                        "SIU-Edwardsville" : "SIU Edwardsville",
                        "St. Francis (NY)" : "St. Francis NY",
                        "St. John's (NY)" : "St. John's",
                        "Tennessee-Martin" : "Tennessee Martin",
                        "Texas A&M-Corpus Christi" : "Texas A&M Corpus Chris",
                        "Texas-Arlington" : "UT Arlington",
                        "Texas Christian" : "TCU",
                        "Texas-El Paso" : "UTEP",
                        "Texas-Rio Grande Valley" : "UT Rio Grande Valley",
                        "Texas-San Antonio" : "UTSA",
                        "Virginia Commonwealth" : "VCU",
                        "UConn" : "Connecticut",
                        "UMass-Lowell" : "UMass Lowell",
                        "UT-Martin" : "Tennessee Martin",
                        "UNC" : "North Carolina",
                        "Pitt" : "Pittsburgh"
                     }

In [2]:
from datetime import datetime
from datetime import timedelta
from sportsreference.ncaab.boxscore import Boxscores
from sportsreference.ncaab.teams import Teams
import pandas as pd
import csv
import os
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait # available since 2.4.0
from selenium.webdriver.support import expected_conditions as EC # available since 2.26.0
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from csv import reader
from csv import writer

In [3]:
options = webdriver.ChromeOptions() 
#options.add_argument(r'''user-data-dir=C:\Users\Tarik's PC\AppData\Local\Google\Chrome\User Data''')
#"user-data-dir=C:\Users\Tarik's PC\AppData\Local\Google\Chrome\User Data" home
# "user-data-dir=C:\Users\koric1\AppData\Local\Google\Chrome\User Data" work
#options.add_argument("--start-maximized")
#self.options.add_argument("--headless")
#self.options.add_argument("--no-sandbox")
#self.options.add_argument("--disable-gpu")
options.add_argument('--headless')
options.add_argument('--disable-gpu')  # Last I checked this was necessa
driver = webdriver.Chrome(chrome_options= options)
driver.get('https://sportsbook.draftkings.com/leagues/basketball/3230960?category=game-lines&subcategory=game')
team_name = str(driver.find_element_by_xpath('//*[@id="root"]/section/section[2]/section/div[3]/div/div[3]/div/div/div[2]/div/div[2]/div[2]/section/div[2]/div[1]/div[1]/a/div/div[2]/span').text)
print(team_name)         

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\urllib3\connectionpool.py", line 377, in _make_request
    httplib_response = conn.getresponse(buffering=True)
TypeError: getresponse() got an unexpected keyword argument 'buffering'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\IPython\core\interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-279010b24e5c>", line 12, in <module>
    driver.get('https://sportsbook.draftkings.com/leagues/basketball/3230960?category=game-lines&subcategory=game')
  File "C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 333, in get
    self.execute(Command.GET, {'url': url})
  File "C

TypeError: must be str, not list

In [4]:
def get_today():
    today = datetime.today()
    year = today.year
    month = today.month
    day = today.day
    return year,month,day

def get_yesterday():
    today = datetime.today()
    yesterday = today - timedelta(days = 1)
    year = yesterday.year
    month = yesterday.month
    day = yesterday.day
    return year,month,day

def get_games(year, month ,day):
    home_teams = []
    away_teams = []
    games = Boxscores(datetime(year, month, day))
    all_games = games.games[str(month)+'-'+str(day)+'-'+str(year)]
    for game in all_games:
        home_teams.append(game['home_name'])
        away_teams.append(game['away_name'])
        
    name = str(year)+"_"+str(month)+"_"+str(day)+"_games.csv"
    with open(name, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["home_team","away_team"])
        for i,team in enumerate(home_teams):
            writer.writerow([home_teams[i],away_teams[i]])
            
    path = os.getcwd()
    os.rename(path+"\\"+name, path+"\\"+"games\\"+name)
    
    return home_teams,away_teams


def get_scores(year, month ,day):
    home_teams = []
    away_teams = []
    home_scores = []
    away_scores = []
    
    games = Boxscores(datetime(year, month, day))
    all_games = games.games[str(month)+'-'+str(day)+'-'+str(year)]
    
    for game in all_games:
        home_teams.append(game['home_name'])
        away_teams.append(game['away_name'])
        home_scores.append(game['home_score'])
        away_scores.append(game['away_score'])
    
    name = str(year)+"_"+str(month)+"_"+str(day)+"_scores.csv"
    with open(name, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["home_team","away_team","home_score","away_score"])
        for i,team in enumerate(home_teams):
            writer.writerow([home_teams[i],away_teams[i],home_scores[i],away_scores[i]])
            
    path = os.getcwd()
    os.rename(path+"\\"+name, path+"\\"+"scores\\"+name)       
    return home_teams,away_teams,home_scores,away_scores

def kenpom(year, month ,day):
    options = webdriver.ChromeOptions() 
    #options.add_argument(r'''user-data-dir=C:\Users\Tarik's PC\AppData\Local\Google\Chrome\User Data''')
    #"user-data-dir=C:\Users\Tarik's PC\AppData\Local\Google\Chrome\User Data" home
    # "user-data-dir=C:\Users\koric1\AppData\Local\Google\Chrome\User Data" work
    #options.add_argument("--start-maximized")
    #self.options.add_argument("--headless")
    #self.options.add_argument("--no-sandbox")
    #self.options.add_argument("--disable-gpu")
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')  # Last I checked this was necessa
    driver = webdriver.Chrome(chrome_options= options)


    name = str(year)+"_"+str(month)+"_"+str(day)+"_kenpom.csv"

    with open(name, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["team","off","def","tempo","luck"])
        #writer.writerow(["SN", "Name", "Contribution"])
        driver.get('https://kenpom.com/')
        time.sleep(1)

        for x in range(1,10):
            try:
                for z in range(1,41):

                    team_name = str(driver.find_element_by_xpath('//*[@id="ratings-table"]/tbody['+str(x)+']/tr['+str(z)+']/td[2]/a').text)
                    team_off = float(driver.find_element_by_xpath('//*[@id="ratings-table"]/tbody['+str(x)+']/tr['+str(z)+']/td[6]').text)
                    team_def = float(driver.find_element_by_xpath('//*[@id="ratings-table"]/tbody['+str(x)+']/tr['+str(z)+']/td[8]').text)
                    team_t = float(driver.find_element_by_xpath('//*[@id="ratings-table"]/tbody['+str(x)+']/tr['+str(z)+']/td[10]').text)
                    team_luck = float(driver.find_element_by_xpath('//*[@id="ratings-table"]/tbody['+str(x)+']/tr['+str(z)+']/td[12]').text)
                    #print(team_name,team_off,team_def,team_t,team_luck)
                    writer.writerow([team_name,team_off,team_def,team_t,team_luck])
            except:
                break
    path = os.getcwd()
    os.rename(path+"\\"+name, path+"\\"+"kenpom\\"+name)
    driver.close()
    

In [5]:
#combine for model
def score_kenmpom(year,month,day):
    path = os.getcwd() 
    scores = path+"\\scores\\"+str(year)+"_"+str(month)+"_"+str(day)+"_scores.csv"
    kenpom =  path+"\\kenpom\\"+str(year)+"_"+str(month)+"_"+str(day)+"_kenpom.csv"
    name = path+"\\scores_kenpom\\"+str(year)+"_"+str(month)+"_"+str(day)+"_scores_kenpom.csv"
    with open(name, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["home","away","home_score","away_score","home_off","home_def","home_tempo","home_luck","away_off","away_def","away_tempo","away_luck"])
        with open(scores, 'r') as read_obj:
            csv_reader = reader(read_obj)
            header = next(csv_reader)
            if header != None:
                for row in csv_reader:
                    home_team = row[0]
                    away_team = row[1]
                    home_score = row[2]
                    away_score = row[3]

                    away_off = 0
                    away_def = 0
                    away_tempo = 0
                    away_luck = 0
                    
                    home_off = 0
                    home_def = 0
                    home_tempo = 0
                    home_luck = 0
                                    
                    

                    if home_team in map_names_not_same.keys():
                        home_team = map_names_not_same[home_team]
                    else:
                        name_list = home_team.split()
                        if name_list[-1] == "State":
                            name_list[-1] = "St."
                            new_name = " ".join(str(x) for x in name_list)
                            home_team = new_name
                        
                    
                    if away_team in map_names_not_same.keys():
                        away_team = map_names_not_same[away_team]
                    else:
                        name_list = away_team.split()
                        if name_list[-1] == "State":
                            name_list[-1] = "St."
                            new_name = " ".join(str(x) for x in name_list)
                            away_team = new_name
                    
                    print(home_team, away_team)

                    with open(kenpom, 'r') as read_obj2:
                        csv_reader2 = reader(read_obj2)
                        header2 = next(csv_reader2)
                        if header2 != None:
                            for row2 in csv_reader2:
                                if row2[0] == home_team:
                                    home_off = row2[1]
                                    home_def = row2[2]
                                    home_tempo = row2[3]
                                    home_luck = row2[4]
                                    break
                    with open(kenpom, 'r') as read_obj2:
                        csv_reader2 = reader(read_obj2)
                        header2 = next(csv_reader2)
                        if header2 != None:
                            for row2 in csv_reader2:
                                if row2[0] == away_team:
                                    away_off = row2[1]
                                    away_def = row2[2]
                                    away_tempo = row2[3]
                                    away_luck = row2[4]
                                    break
                    if home_off == 0 or away_off == 0:
                        continue
                    writer.writerow([home_team,away_team,home_score,away_score,home_off,home_def,home_tempo,home_luck,away_off,away_def,away_tempo,away_luck])
      
    

In [6]:
#add to all 
def add_to_all(year,month,day):
    path = os.getcwd() 
    name = path+"\\scores_kenpom\\"+str(year)+"_"+str(month)+"_"+str(day)+"_scores_kenpom.csv"
    date = str(year)+"-"+str(month)+"-"+str(day)
    with open("all.csv", 'a+', newline='') as write_obj:
        csv_writer = writer(write_obj)
        with open(name, 'r') as read_obj:
            csv_reader = reader(read_obj)
            header = next(csv_reader)
            if header != None:
                for row in csv_reader:
                    print(row[0],row[1],row[2],row[3],row[4],row[5],row[6],row[7],row[8],row[9],row[10],row[11],date)
                    csv_writer.writerow([row[0],row[1],row[2],row[3],row[4],row[5],row[6],row[7],row[8],row[9],row[10],row[11],date])

In [7]:
year,month,day = get_today()

In [7]:
year,month,day = get_today()
kenpom(year, month ,day)
get_games(year, month ,day)

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\ipykernel_launcher.py:75: DeprecationWarning: use options instead of chrome_options


(['Arizona State',
  'Buffalo',
  'Northern Arizona',
  'Nicholls State',
  'Stetson',
  'Campbell',
  'Western Kentucky',
  'Ole Miss',
  'San Francisco',
  'VMI',
  'Minnesota',
  'Western Illinois',
  'Jacksonville',
  'Coppin State',
  'South Dakota',
  'South Dakota State',
  'East Carolina',
  'Oregon State',
  'Ohio',
  'Texas A&M-Corpus Christi',
  'Morehead State',
  'North Alabama'],
 ['San Diego State',
  '',
  'UC-Riverside',
  'Campbellsville',
  'Florida College',
  'Florida National University',
  'Gardner-Webb',
  'Jackson State',
  'Long Beach State',
  'Mercer',
  'UMKC',
  'Mt. St. Joseph',
  'New Orleans',
  'UNC Greensboro',
  'North Dakota',
  'North Dakota State',
  'North Florida',
  'Portland',
  'Purdue Northwest',
  'Texas-Rio Grande Valley',
  'Transylvania',
  'Troy'])

In [8]:
year,month,day = get_yesterday()
get_scores(year, month ,day)

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\koric1\\Documents\\college_basketball\\2020_12_10_scores.csv' -> 'C:\\Users\\koric1\\Documents\\college_basketball\\scores\\2020_12_10_scores.csv'

In [9]:
score_kenmpom(year,month,day)

Arizona St. San Diego St.
Northern Arizona UC Riverside
Campbell Florida National University
Western Kentucky Gardner Webb
Ole Miss Jackson St.
San Francisco Long Beach St.
Minnesota UMKC
Western Illinois Mt. St. Joseph
Jacksonville New Orleans
Coppin St. UNC Greensboro
South Dakota North Dakota
South Dakota St. North Dakota St.
East Carolina North Florida
Oregon St. Portland
Ohio Purdue Northwest
Texas A&M Corpus Chris UT Rio Grande Valley
Morehead St. Transylvania
North Alabama Troy


In [10]:
add_to_all(year,month,day)

Arizona St. San Diego St. 68 80 111.3 92.8 75.8 0.052 107.0 89.5 68.6 0.035 2020-12-10
Northern Arizona UC Riverside 50 74 94.4 106.6 70.1 0.0 95.7 96.8 68.9 -0.089 2020-12-10
Western Kentucky Gardner Webb 86 84 107.3 96.8 70.6 0.06 95.8 103.2 70.8 0.0 2020-12-10
San Francisco Long Beach St. 107 62 105.1 94.3 72.0 -0.007 96.2 102.7 75.0 0.178 2020-12-10
Minnesota UMKC 90 61 107.3 91.1 73.4 0.098 93.5 103.2 69.0 0.0 2020-12-10
Jacksonville New Orleans 77 70 92.0 102.9 72.6 0.011 92.6 105.5 72.8 -0.093 2020-12-10
Coppin St. UNC Greensboro 85 80 87.0 102.3 75.9 -0.03 102.5 95.7 72.4 -0.005 2020-12-10
South Dakota North Dakota 71 75 97.5 102.9 71.7 -0.032 97.1 104.6 71.1 -0.112 2020-12-10
South Dakota St. North Dakota St. 77 75 108.6 98.5 72.1 0.045 96.8 98.6 68.2 -0.035 2020-12-10
East Carolina North Florida 73 67 100.7 96.0 71.1 0.001 97.7 109.4 71.8 -0.09 2020-12-10
Oregon St. Portland 86 87 102.5 98.2 69.7 -0.185 92.8 102.4 74.3 0.005 2020-12-10
Texas A&M Corpus Chris UT Rio Grande Val

In [11]:
def predict(year,month,day):
    path = os.getcwd() 
    games = path+"\\games\\"+str(year)+"_"+str(month)+"_"+str(day)+"_games.csv"
    name = path+"\\predict\\"+str(year)+"_"+str(month)+"_"+str(day)+"_predict.csv"
    kenpom = path+"\\kenpom\\"+str(year)+"_"+str(month)+"_"+str(day)+"_kenpom.csv"
    with open(name, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["home","away","home_off","home_def","home_tempo","home_luck","away_off","away_def","away_tempo","away_luck"])
        with open(games, 'r') as read_obj:
            csv_reader = reader(read_obj)
            header = next(csv_reader)
            if header != None:
                for row in csv_reader:
                    home_team = row[0]
                    away_team = row[1]

                    away_off = 0
                    away_def = 0
                    away_tempo = 0
                    away_luck = 0
                    
                    home_off = 0
                    home_def = 0
                    home_tempo = 0
                    home_luck = 0
                

                    if home_team in map_names_not_same.keys():
                        home_team = map_names_not_same[home_team]
                    else:
                        name_list = home_team.split()
                        if name_list[-1] == "State":
                            name_list[-1] = "St."
                            new_name = " ".join(str(x) for x in name_list)
                            home_team = new_name
                        
                    
                    if away_team in map_names_not_same.keys():
                        away_team = map_names_not_same[away_team]
                    else:
                        name_list = away_team.split()
                        if name_list[-1] == "State":
                            name_list[-1] = "St."
                            new_name = " ".join(str(x) for x in name_list)
                            away_team = new_name


                    with open(kenpom, 'r') as read_obj2:
                        csv_reader2 = reader(read_obj2)
                        header2 = next(csv_reader2)
                        if header2 != None:
                            for row2 in csv_reader2:
                                if row2[0] == home_team:
                                    home_off = row2[1]
                                    home_def = row2[2]
                                    home_tempo = row2[3]
                                    home_luck = row2[4]
                                    break
                    with open(kenpom, 'r') as read_obj2:
                        csv_reader2 = reader(read_obj2)
                        header2 = next(csv_reader2)
                        if header2 != None:
                            for row2 in csv_reader2:
                                if row2[0] == away_team:
                                    away_off = row2[1]
                                    away_def = row2[2]
                                    away_tempo = row2[3]
                                    away_luck = row2[4]
                                    break
                    if home_off == 0 or away_off == 0:
                        continue
                    writer.writerow([home_team,away_team,home_off,home_def,home_tempo,home_luck,away_off,away_def,away_tempo,away_luck])

In [12]:
year,month,day = get_today()

In [13]:
predict(year, month,day)

In [101]:
score_kenmpom(2020,2,19)

N.C. State Duke
Louisville Syracuse
DePaul Villanova
Georgia Auburn
Seton Hall Butler
Houston Tulsa
Houston Baptist Abilene Christian
Fresno St. Air Force
Loyola Marymount Army
San Jose St. Boise St.
Virginia Boston College
Lafayette Boston University
Washington St. California
UMKC Cal Baptist
Stephen F. Austin Central Arkansas
Cincinnati UCF
The Citadel Chattanooga
Memphis East Carolina
La Salle Fordham
ETSU Furman
Richmond George Mason
Duquesne George Washington
Wake Forest Georgia Tech
Loyola Chicago Illinois St.
Minnesota Indiana
Siena Iona
Texas Tech Kansas St.
Sam Houston St. Lamar
Colgate Lehigh
UC Irvine Long Beach St.
Arkansas St. Louisiana Monroe
Incarnate Word McNeese St.
Virginia Tech Miami FL
Rutgers Michigan
Bradley Missouri St.
American Navy
Northwestern St. Nicholls St.
South Dakota North Dakota St.
Georgetown Providence
Mercer Samford
Mississippi St. South Carolina
North Dakota South Dakota St.
Tulane SMU
Alabama Texas A&M
Texas TCU
Drake Valparaiso
Western Carolina VM

In [63]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.ensemble import AdaBoostRegressor
from sklearn import linear_model

In [51]:
df = pd.read_csv('all.csv')
df['line'] = df['home_score'] - df['away_score']
df.head()
datetime_object = datetime.strptime("2/10/2020", '%m/%d/%Y').date()


In [53]:
df['date']= pd.to_datetime(df['date']) 
df.info() 
df = df[df['date'] > datetime_object]
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 553 entries, 0 to 552
Data columns (total 14 columns):
home          553 non-null object
away          553 non-null object
home_score    553 non-null int64
away_score    553 non-null int64
home_off      553 non-null float64
home_def      553 non-null float64
home_tempo    553 non-null float64
home_luck     553 non-null float64
away_off      553 non-null float64
away_def      553 non-null float64
away_tempo    553 non-null float64
away_luck     553 non-null float64
date          553 non-null datetime64[ns]
line          553 non-null int64
dtypes: datetime64[ns](1), float64(8), int64(3), object(2)
memory usage: 60.6+ KB


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  This is separate from the ipykernel package so we can avoid doing imports until


,home,away,home_score,away_score,home_off,home_def,home_tempo,home_luck,away_off,away_def,away_tempo,away_luck,date,line
196,San Diego St.,New Mexico,82,59,115.1,90.6,64.6,-0.008,105.3,102.4,71.8,0.044,2020-02-11,23
197,Dayton,Rhode Island,81,67,119.1,94.1,67.6,0.002,105.2,94.2,71.2,0.068,2020-02-11,14
198,Maryland,Nebraska,72,70,113.5,92.4,67.0,0.037,102.0,101.3,73.1,-0.047,2020-02-11,2
199,Vanderbilt,Kentucky,64,78,104.7,104.5,69.4,-0.061,112.7,95.1,67.7,0.018,2020-02-11,-14
200,Purdue,Penn St.,76,88,109.3,90.8,64.6,-0.104,110.3,92.4,70.9,0.014,2020-02-11,-12


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 553 entries, 0 to 552
Data columns (total 14 columns):
home          553 non-null object
away          553 non-null object
home_score    553 non-null int64
away_score    553 non-null int64
home_off      553 non-null float64
home_def      553 non-null float64
home_tempo    553 non-null float64
home_luck     553 non-null float64
away_off      553 non-null float64
away_def      553 non-null float64
away_tempo    553 non-null float64
away_luck     553 non-null float64
date          553 non-null datetime64[ns]
line          553 non-null int64
dtypes: datetime64[ns](1), float64(8), int64(3), object(2)
memory usage: 60.6+ KB


In [94]:
def models(year,month,day,parse = False):
    #read df
    df = pd.read_csv('all.csv')
    df['line'] = df['home_score'] - df['away_score']
    
    #parse by date 
    if parse == True:
        datetime_object = datetime.strptime( day+"/"+month+"/"+year, '%m/%d/%Y').date()
        df['date']= pd.to_datetime(df['date']) 
        df = df[df['date'] > datetime_object]
    
    # get X and y data
    X = df[['home_off','home_def','home_tempo','home_luck','away_off','away_def','away_tempo','away_luck']].values
    y = df[['line']].values 
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)
    
    #linear 
    model_lin = LinearRegression()  
    model_lin.fit(X_train, y_train)
    
    #random forrest
    model_rf = RandomForestRegressor()
    model_rf.fit(X_train, y_train)
    
    #ridge
    model_ridge = Ridge(alpha= .1)
    model_ridge.fit(X_train, y_train)
    
    #lasso
    model_lasso = Lasso(alpha=0.1)
    model_lasso.fit(X_train, y_train)
    
    #adaboost
    model_ada = AdaBoostRegressor(learning_rate = 1)
    model_ada.fit(X_train, y_train)
    
    return model_lin,model_rf,model_ridge,model_lasso,model_ada
    

In [109]:
def predict_scores(year,month,day,model_lin,model_rf,model_ridge,model_lasso,model_ada):
    path = os.getcwd()
    predict_csv = path+"\\predict\\"+str(year)+"_"+str(month)+"_"+str(day)+"_predict.csv"
    name = path+"\\predict_scores\\"+str(year)+"_"+str(month)+"_"+str(day)+"_predict_scores.csv"
    games = path+"\\games\\"+str(year)+"_"+str(month)+"_"+str(day)+"_games.csv"
    df = pd.read_csv(predict_csv)
    
    X = df[['home_off','home_def','home_tempo','home_luck','away_off','away_def','away_tempo','away_luck']].values
    
    pred_lin = model_lin.predict(X).round()
    pred_rf = model_rf.predict(X).round()
    pred_ridge = model_ridge.predict(X).round()
    pred_lasso = model_lasso.predict(X).round()
    pred_ada = model_ada.predict(X).round()
    
    with open(name, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["home","away","lin","rf","ridge","lasso","ada","average"])
        with open(games, 'r') as read_obj:
            csv_reader = reader(read_obj)
            header = next(csv_reader)
            if header != None:
                for i,row in enumerate(csv_reader):
                    average = ((pred_lin[i][0]+pred_rf[i]+pred_ridge[i][0]+pred_lasso[i]+pred_ada[i])/5).round()
                    writer.writerow([row[0],row[1],pred_lin[i][0],pred_rf[i],pred_ridge[i][0],pred_lasso[i],pred_ada[i],average])
                    
    
    

In [117]:
def predict_scores_error(year,month,day):
    path = os.getcwd()
    predict_scores = path+"\\predict_scores\\"+str(year)+"_"+str(month)+"_"+str(day)+"_predict_scores.csv"
    name = path+"\\predict_error\\"+str(year)+"_"+str(month)+"_"+str(day)+"_predict_error.csv"
    scores = path+"\\scores\\"+str(year)+"_"+str(month)+"_"+str(day)+"_scores.csv"
    
    df = pd.read_csv(predict_scores)
    y = df[["home","away","lin","rf","ridge","lasso","ada","average"]].values
    
    df2 = pd.read_csv(scores)
    df2['line'] = df2['home_score'] - df2['away_score']
    y2 = df2[["line"]].values
    
    with open(name, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["home","away","lin","rf","ridge","lasso","ada","average","actual"])
        for i,row in enumerate(y):
            writer.writerow([row[0],row[1],row[2],row[3],row[4],row[5],row[6],row[7],y2[i][0]])
    
    

In [122]:
year = 2020
month = 2
day = 20
get_scores(year, month ,day)
get_games(year, month ,day)
predict(year,month,day)
model_lin,model_rf,model_ridge,model_lasso,model_ada = models(year,month,day,parse = False)
predict_scores(year,month,day,model_lin,model_rf,model_ridge,model_lasso,model_ada)
predict_scores_error(year,month,day)

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\ipykernel_launcher.py:24: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [22]:
X = df[['home_off','home_def','home_tempo','home_luck','away_off','away_def','away_tempo','away_luck']].values
y = df[['line']].values

In [97]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#linear regression
model_lin = LinearRegression()  
model_lin.fit(X_train, y_train)
y_pred = model_lin.predict(X_test)
model_lin_mae = mean_absolute_error(y_test, y_pred)

#random forrest
model_rf = RandomForestRegressor()
model_rf.fit(X_train, y_train)
y_pred = model_rf.predict(X_test)
model_rf_mae = mean_absolute_error(y_test, y_pred)

#ridge
model_ridge = Ridge(alpha= .1)
model_ridge.fit(X_train, y_train)
y_pred = model_ridge.predict(X_test)
model_ridge_mae = mean_absolute_error(y_test, y_pred)

#lasso
clf = linear_model.Lasso(alpha=0.1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
model_lasso_mae = mean_absolute_error(y_test, y_pred)

#adaboast
regr = AdaBoostRegressor(learning_rate = 1)
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)
model_ada_mae = mean_absolute_error(y_test, y_pred)

print(model_lin_mae,model_rf_mae,model_ridge_mae, model_lasso_mae,model_ada_mae)

7.720314738552142 9.377477477477479 7.718427959911215 7.701115991647981 8.709523004075026


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\ipykernel_launcher.py:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # This is added back by InteractiveShellApp.init_path()
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
def models(year,month,day):
    

In [28]:
clf.coef_

array([ 0.72102173, -0.73691876,  0.05610929, -0.        , -0.7604621 ,
        0.72835937,  0.01465226, -0.        ])

In [29]:
model_lin.coef_

array([[ 0.72557169, -0.74121843,  0.08249751, -4.85974325, -0.76751941,
         0.73212717,  0.02083887, -7.81824327]])

In [19]:
df2 = pd.read_csv('predict\\2020_2_20_predict.csv')
df2

,home,away,home_off,home_def,home_tempo,home_luck,away_off,away_def,away_tempo,away_luck
0,Gonzaga,San Francisco,121.3,94.4,71.9,0.050,108.3,98.2,68.7,-0.006
1,Arizona St.,Oregon,106.0,94.7,73.2,0.069,117.1,97.1,65.0,0.023
2,Colorado,USC,109.1,93.4,68.2,-0.010,104.3,92.0,68.9,0.083
3,Iowa,Ohio St.,117.3,98.6,70.2,0.005,114.3,92.0,66.2,-0.038
4,BYU,Santa Clara,116.3,95.6,69.5,-0.028,102.6,101.3,71.6,0.049
5,Arizona,Oregon St.,111.0,91.4,69.3,-0.077,110.9,102.0,65.6,-0.002
6,SIU-Edwardsville,Austin Peay,111.0,91.4,69.3,-0.077,106.9,107.0,67.8,0.032
7,Morehead St.,Belmont,94.6,108.1,67.3,-0.001,106.5,100.5,70.5,0.012
8,Utah Valley,Cal St. Bakersfield,96.5,104.2,70.4,-0.092,96.2,104.9,64.9,-0.063
9,Cal Poly,UC Davis,95.2,108.5,68.3,-0.038,104.7,109.4,68.0,-0.024


In [20]:
X = df2[['home_off','home_def','home_tempo','home_luck','away_off','away_def','away_tempo','away_luck']].values
y_pred = model_lin.predict(X).round()
y_pred

array([[ 16.],
       [ -3.],
       [  6.],
       [  1.],
       [ 18.],
       [ 12.],
       [ 18.],
       [-11.],
       [  6.],
       [ -2.],
       [  1.],
       [  0.],
       [ 15.],
       [  2.],
       [ 11.],
       [ -1.],
       [  7.],
       [  4.],
       [  7.],
       [  3.],
       [  3.],
       [ -6.],
       [ 19.],
       [  4.],
       [  3.],
       [ 17.],
       [  3.],
       [ -3.],
       [-13.],
       [  7.],
       [ -2.],
       [  0.],
       [ 11.],
       [  9.],
       [  2.],
       [ -4.],
       [ -4.],
       [ -8.],
       [ -6.],
       [ -6.],
       [  2.],
       [  8.],
       [  2.],
       [ 12.],
       [ 11.],
       [ 11.],
       [  4.],
       [  3.],
       [ -2.],
       [  3.],
       [ 11.],
       [ -0.],
       [ -5.],
       [  5.],
       [ -3.]])

In [13]:
with open("test.csv", 'w', newline='') as file:
        writer = csv.writer(file)
        for row in y_pred:
            writer.writerow([row[0],row[1]])

In [12]:
get_scores(2020, 2,20)

(['Gonzaga',
  'Arizona State',
  'Colorado',
  'Iowa',
  'BYU',
  'Arizona',
  'SIU-Edwardsville',
  'Morehead State',
  'Utah Valley',
  'Cal Poly',
  'Cal State Fullerton',
  'Charleston Southern',
  'Temple',
  'James Madison',
  'Southern Illinois',
  'Hampton',
  'Texas State',
  'Texas-Arlington',
  'Seattle',
  'New Hampshire',
  'UC-Riverside',
  'Drexel',
  'North Alabama',
  'Jacksonville',
  'USC Upstate',
  "Saint Mary's",
  'Albany (NY)',
  'Binghamton',
  'Nebraska',
  'Portland State',
  'Eastern Illinois',
  'Purdue-Fort Wayne',
  'Radford',
  'Liberty',
  'Delaware',
  'Idaho',
  'Indiana State',
  'Cleveland State',
  'Denver',
  'Portland',
  'High Point',
  'Eastern Washington',
  'Appalachian State',
  'Wichita State',
  'Jacksonville State',
  'Northern Colorado',
  'Washington',
  'NJIT',
  'Tennessee Tech',
  'Eastern Kentucky',
  'Coastal Carolina',
  'Utah',
  'Stony Brook',
  'Towson',
  'Youngstown State'],
 ['San Francisco',
  'Oregon',
  'USC',
  'Ohio St

In [39]:
add_to_all(2020,2,7)

In [32]:

score_kenmpom(2020,2,7)


In [38]:
year,month,day = get_yesterday()
print(year, month, day)

2020 9 22


In [36]:
type(year)

int

In [37]:
home_teams,away_teams,home_scores,away_scores = get_scores(2020, 2,7)

In [8]:
for i, val in enumerate(home_teams):
    print(home_scores[i],home_teams[i],away_teams[i],away_scores[i])

NameError: name 'home_teams' is not defined

In [2]:
kp = pd.read_csv('kenpom.csv')
kp.head()

,team,off,def,tempo,luck
0,Abilene Christian,99.1,100.2,68.4,-0.023
1,Air Force,107.8,110.3,68.8,-0.059
2,Akron,108.5,98.9,69.4,0.022
3,Alabama,111.0,99.5,74.8,-0.076
4,Alabama A&M,87.9,108.2,67.7,0.014


In [3]:
for index, row in kp.iterrows():
    name = row['team']
    name_list = name.split()
    if name_list[-1] == "St.":
        name_list[-1] = "State"
        new_name = " ".join(str(x) for x in name_list)
        kp['team'][index] = new_name
        
        
    

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [4]:
teams_ken_pom = kp['team'].tolist()

In [5]:
teams = Teams()
teams_sportsreference = []
for team in teams:
    teams_sportsreference.append(team.name)

In [11]:
for team in teams_sportsreference:
    if team not in teams_ken_pom:
        if team not in map_names_not_same.keys():
            print(team)

In [88]:
home_teams,away_teams,home_scores,away_scores = get_scores(2020, 3 ,8)

In [89]:
h_off = []
h_def = []
h_tempo = []
h_luck = []
a_off = []
a_def = []
a_tempo = []
a_luck = []
not_in_h = []
not_in_a = []
for i,team in enumerate(home_teams):
    check = False
    for index, row in kp.iterrows():
        if row['team'] == team:
            h_off.append(row['off'])
            h_def.append(row['def'])
            h_tempo.append(row['tempo'])
            h_luck.append(row['luck'])
            check = True
    if not check:
        h_off.append(None)
        h_def.append(None)
        h_tempo.append(None)
        h_luck.append(None)
        not_in_h.append(i)

for i,team in enumerate(away_teams):
    check = False
    for index, row in kp.iterrows():
        if row['team'] == team and i not in not_in_h:
            a_off.append(row['off'])
            a_def.append(row['def'])
            a_tempo.append(row['tempo'])
            a_luck.append(row['luck'])
            check = True
    if not check:
        not_in_a.append(i)
x = 0
for i in not_in_a:
    num = i - x
    del h_off[num]
    del h_def[num]
    del h_tempo[num]
    del h_luck[num]
    del away_teams[num]
    del home_teams[num]
    x += 1
    

In [90]:
len(h_off)

17

In [91]:
len(home_teams)

17

In [93]:
for i, val in enumerate(home_teams):
    print(home_teams[i],away_teams[i], h_off[i], a_off[i], i + 1)

Maryland Michigan 113.5 113.2 1
Michigan State Ohio State 115.2 114.3 2
Illinois Iowa 110.9 117.3 3
Houston Memphis 112.7 100.6 4
Boston University Bucknell 103.3 96.2 5
Hofstra Drexel 108.7 100.0 6
UCF East Carolina 103.5 99.6 7
William & Mary Elon 103.3 100.6 8
Winthrop Hampton 104.7 104.8 9
Colgate Lafayette 107.2 100.6 10
Liberty Lipscomb 106.1 101.2 11
Minnesota Nebraska 112.1 102.0 12
South Dakota North Dakota 108.6 102.2 13
Towson Northeastern 105.7 105.4 14
Wichita State Tulsa 105.6 103.2 15
Bradley Valparaiso 106.8 104.5 16
Chattanooga Wofford 106.6 104.5 17


In [64]:
for i, val in enumerate(home_teams):
    print(home_scores[i],home_teams[i],away_teams[i],away_scores[i], i)

83 Maryland Michigan 70 0
80 Michigan State Ohio State 69 1
78 Illinois Iowa 76 2
64 Houston Memphis 57 3
64 Boston University Bucknell 61 4
76 Tulane UConn 80 5
67 College of Charleston Delaware 79 6
61 Hofstra Drexel 43 7
94 UCF East Carolina 62 8
63 William & Mary Elon 68 9
76 Winthrop Hampton 68 10
89 Colgate Lafayette 64 11
73 Liberty Lipscomb 57 12
107 Minnesota Nebraska 75 13
79 Oral Roberts Omaha 52 14
71 South Dakota North Dakota 74 15
62 Towson Northeastern 72 16
79 Wichita State Tulsa 57 17
80 Bradley Valparaiso 66 18
97 ETSU Western Carolina 75 19
70 Chattanooga Wofford 72 20
